In [35]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [36]:
symbol = 'spy'
data = vbt.AlpacaData.download(symbol,
                               start='1990-3-05',
                               end='2023-3-16',
                               limit=2000000,
                               timeframe='1d').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


FigureWidget({
    'data': [{'close': array([185.59, 183.69, 181.12, ..., 390.22, 387.78, 394.59]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([185.71, 185.87, 184.24, ..., 391.94, 387.99, 394.94]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([184.21, 183.43, 180.37, ..., 385.56, 382.23, 384.8 ]),
              'name': 'Candlestick',
              'open': array([184.5 , 185.53, 183.96, ..., 389.  , 384.41, 385.33]),
              'type': 'candlestick',
              'uid': '40ce195d-2c9b-4dfe-acb9-0564d5a12f4c',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 14, 4, 0, tzinfo=datetime.timezone.utc),
             

In [37]:
RSI = vbt.IndicatorFactory.from_talib('RSI')
SMA = vbt.IndicatorFactory.from_talib('SMA')


def pullback_place_entry(start, close, rsi_window = 10, sma_window = 200, entry = 30, out = 40, leave = 10):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    sma = SMA.run(close, sma_window).real.to_numpy()

    entry = np.where((rsi < entry) & (sma < start), 1, 0)

    entry = np.roll(entry, 2)

    leave_array = np.roll(entry.astype(float), leave)
    leave_array[:leave] = np.inf


    sell = np.where((leave_array == 1) | (rsi>out) , -1, 0)
    sell = np.roll(sell, 2)

    exits = sell == -1
    entry = order.value == 1

    return entry, exits

PULLBACK_PLACE_ENTRY = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['start', 'close'],
    output_names = ['entry', 'exits'],
    param_names=['rsi_window', 'sma_window', 'entry', 'out', 'leave']
).from_apply_func(pullback_place_entry, rsi_window = 10, sma_window = 200, entry = 30, out = 40, leave=10)

order = PULLBACK_PLACE_ENTRY.run(
    data['Open'],
    data['Close'],
    #rsi_window = np.arange(10,30,step=1,dtype=int),
    sma_window = np.arange(50,300,step=5,dtype=int),
    entry = np.arange(10,50,step=2,dtype=int),
    out= np.arange(10,70,step=2,dtype=int),
    #leave=np.arange(5,15,step=1,dtype=int),
    param_product = True,
    to_2d = False
)




pf = vbt.Portfolio.from_signals(data['Open'], order.entries, order.exits, init_cash=10000, freq='1d')

#orders = pf.orders

returns = pf.total_return()
print(returns)
print(returns.max())
'''
fig = returns.vbt.volume(
        x_level = "pullback_place_order_rsi_window",
        y_level = "pullback_place_order_sma_window",
        z_level = "pullback_place_order_out",
        trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%'))
)
fig.show()'''

,Open,High,Low,Close,Volume
timestamp,,,,,
2015-12-01 05:00:00+00:00,184.50,185.71,184.21,185.59,97858418.0
2015-12-02 05:00:00+00:00,185.53,185.87,183.43,183.69,108069059.0
2015-12-03 05:00:00+00:00,183.96,184.24,180.37,181.12,166224154.0
2015-12-04 05:00:00+00:00,181.54,184.96,181.40,184.66,192878747.0
2015-12-07 05:00:00+00:00,184.31,184.75,182.52,183.54,102027111.0
2015-12-08 05:00:00+00:00,181.90,183.48,181.27,182.30,103372367.0
2015-12-09 05:00:00+00:00,181.63,183.83,179.86,180.89,162401537.0
2015-12-10 05:00:00+00:00,180.96,182.73,180.71,181.35,116128858.0
2015-12-11 05:00:00+00:00,179.13,179.83,177.51,177.84,211173305.0


pullback_place_order_sma_window  pullback_place_order_entry  pullback_place_order_out
50                               10                          10                          0.00000
                                                             12                          0.00000
                                                             14                          0.00000
                                                             16                          0.00000
                                                             18                          0.00000
                                                                                          ...   
295                              48                          60                          0.36761
                                                             62                          0.36761
                                                             64                          0.36761
                                         

'\nfig = returns.vbt.volume(\n        x_level = "pullback_place_order_rsi_window",\n        y_level = "pullback_place_order_sma_window",\n        z_level = "pullback_place_order_out",\n        trace_kwargs=dict(colorbar=dict(title=\'Total return\', tickformat=\'%\'))\n)\nfig.show()'